In [113]:
import pandas as pd
import numpy as np

In [192]:
df = pd.read_csv('train.csv') 
df1 = df[df['building_number'] == 1]
df2 = df1.drop(['building_number','sub_meter_2',], axis=1)
print(df2.head())
print(df2.tail())
df2['sub_meter_1'] =df2['sub_meter_1'].diff() #np.log(df2['sub_meter_1'])#.diff()
df2['main_meter'] = df2['main_meter'].diff() #np.log(df2['main_meter'])#.diff()

          timestamp   main_meter  sub_meter_1
0  01-04-2017 00:00  2898.969633  1390.244624
1  01-04-2017 00:15  2531.896894  1087.313897
2  01-04-2017 00:30  2171.649138   695.625737
3  01-04-2017 00:45  2207.299159   615.496571
4  01-04-2017 01:00  2839.441067  1071.573744
              timestamp   main_meter  sub_meter_1
26395  31-12-2017 22:45  1940.269619   838.899936
26396  31-12-2017 23:00  2032.948747   871.474029
26397  31-12-2017 23:15  2090.305746   894.499291
26398  31-12-2017 23:30  2007.119194   827.307818
26399  31-12-2017 23:45  2163.604520   854.982388


In [193]:
df2=df2.drop(df2[df['sub_meter_1']==0].index)


c:\users\shivam raj\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [194]:
df2[df['sub_meter_1']==0]

c:\users\shivam raj\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,timestamp,main_meter,sub_meter_1


In [195]:
print(df2.head())
print(df2.tail())

          timestamp  main_meter  sub_meter_1
0  01-04-2017 00:00         NaN          NaN
1  01-04-2017 00:15 -367.072739  -302.930727
2  01-04-2017 00:30 -360.247756  -391.688160
3  01-04-2017 00:45   35.650021   -80.129165
4  01-04-2017 01:00  632.141908   456.077173
              timestamp  main_meter  sub_meter_1
26395  31-12-2017 22:45 -149.471634   -86.212293
26396  31-12-2017 23:00   92.679128    32.574093
26397  31-12-2017 23:15   57.356999    23.025262
26398  31-12-2017 23:30  -83.186552   -67.191473
26399  31-12-2017 23:45  156.485326    27.674571


In [196]:
df2=df2.dropna()

In [197]:
X_train,X_test = df2[120*96:-30*96],df2[-30*96:]

In [198]:
X_train.shape,X_test.shape

((11993, 3), (2880, 3))

In [199]:
X_train.head()

,timestamp,main_meter,sub_meter_1
11521,30-07-2017 00:15,-120.567250,-130.788234
11522,30-07-2017 00:30,-398.292134,-342.043934
11523,30-07-2017 00:45,3.633635,-121.011075
11524,30-07-2017 01:00,307.455900,269.457168
11525,30-07-2017 01:15,173.761289,290.518443


In [200]:
X_train.tail()

,timestamp,main_meter,sub_meter_1
23515,01-12-2017 22:45,94.911472,-96.515682
23516,01-12-2017 23:00,-117.405874,21.379652
23517,01-12-2017 23:15,-173.420914,-60.547507
23518,01-12-2017 23:30,-4.918042,4.386254
23519,01-12-2017 23:45,-103.930573,-96.069635


In [201]:
import statsmodels.api as sm
from statsmodels.tsa.api import VAR

In [202]:
X_train2= X_train.drop(['timestamp'],axis=1)
#X_train2 = np.asarray(X_train2)
X_train2.index = X_train['timestamp']

In [203]:
mod = VAR(X_train2)
res = mod.fit(maxlags=96*14)
print(res.summary())

c:\users\shivam raj\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\shivam raj\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:222: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  ' forecasting.', ValueWarning)


  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 01, Feb, 2020
Time:                     23:13:11
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                    24.6919
Nobs:                     10649.0    HQIC:                   22.2582
Log likelihood:          -136757.    FPE:                1.37413e+09
AIC:                      21.0188    Det(Omega_mle):     8.75919e+08
--------------------------------------------------------------------
Results for equation main_meter
                       coefficient       std. error           t-stat            prob
------------------------------------------------------------------------------------
const                     0.272172         3.782082            0.072           0.943
L1.main_meter            -0.072239         0.012275           -5.885           0.000
L1.sub_meter_1           -0.400517     

In [204]:
X_test2= X_test.drop(['timestamp'],axis=1)
X_test2.index = X_test['timestamp']

In [205]:
prediction = res.forecast(res.y, steps=len(X_test2))

c:\users\shivam raj\appdata\local\programs\python\python37\lib\site-packages\statsmodels\base\wrapper.py:36: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  obj = getattr(results, attr)


In [206]:
prediction

array([[-105.06132497,   -8.3303346 ],
       [ 168.34822504, -149.05416673],
       [-210.23043758,  -95.52480584],
       ...,
       [ 131.40266884,  133.75639296],
       [-315.67674982,   88.13313631],
       [1321.73127927,  152.04000358]])

In [207]:
X_test2.head()


,main_meter,sub_meter_1
timestamp,,
02-12-2017 00:00,-49.527084,-29.346890
02-12-2017 00:15,-105.439221,-86.656423
02-12-2017 00:30,-91.176186,-75.433857
02-12-2017 00:45,340.272522,17.379591
02-12-2017 01:00,191.676692,41.411902


In [208]:
prediction = pd.DataFrame(prediction)
prediction['main_meter'] = prediction[0]

In [209]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(np.asarray(X_test2['main_meter']), np.asarray(prediction['main_meter'])))

c:\users\shivam raj\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [191]:
rms


4103.927305926196